# Live cu-inj-live-impact 

In [1]:
# Setup directories, and convert dashboard notebook to a script for importing
#!./setup.bash

In [2]:
#%load_ext autoreload
#%autoreload 2

In [3]:
from impact import evaluate_impact_with_distgen, run_impact_with_distgen
from impact.tools import isotime
from impact.evaluate import  default_impact_merit
from impact import Impact

from make_dashboard import make_dashboard
from get_vcc_image import get_live_distgen_xy_dist, VCC_DEVICE_PV

import matplotlib as mpl

from pmd_beamphysics.units import e_charge

In [4]:
import pandas as pd
import numpy as np

import h5py
import json
import epics

import sys
import os
from time import sleep, time


import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('Agg')

# Nicer plotting
# %config InlineBackend.figure_format = 'retina'

# Logging

In [5]:
#MODEL = 'f2e_inj'
MODEL = 'cu_inj'

In [6]:
PREFIX = f'lume-impact-live-demo-{MODEL}'

In [7]:
import logging

# Gets or creates a logger
logger = logging.getLogger(PREFIX)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler(f'{PREFIX}.log')
#formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
formatter    = logging.Formatter(fmt="%(asctime)s :  %(name)s : %(message)s ", datefmt="%Y-%m-%dT%H:%M:%S%z")

# Add print to stdout
logger.addHandler(logging.StreamHandler(sys.stdout))

file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

# Configuration

Set up basic input sources and output path.

Depends on the `$LCLS_LATTICE` environmental variable pointing to a checked out copy of https://github.com/slaclab/lcls-lattice (protected)

See README for required environment variables.

In [8]:
DEBUG = False

WORKDIR=os.environ.get('IMPACT_WORKDIR')
if not WORKDIR:
    raise ValueError("IMPACT_WORKDIR variable not set.")


HOST = os.environ.get('IMPACT_HOST') # mcc-simul or 'sdf'
if not HOST:
    raise ValueError("IMPACT_HOST variable not set.")

    
IMPACT_CONFIG_FILE=os.environ.get('IMPACT_CONFIG_FILE')
if not IMPACT_CONFIG_FILE:
    raise ValueError("IMPACT_CONFIG_FILE variable not set.")

DISTGEN_INPUT_FILE=os.environ.get('IMPACT_DISTGEN_INPUT_FILE')
if not DISTGEN_INPUT_FILE:
    raise ValueError("IMPACT_DISTGEN_INPUT_FILE variable not set.")
    
# Directory for summary output
SUMMARY_OUTPUT_DIR = os.environ.get('IMPACT_SUMMARY_OUTPUT_DIR')
if not SUMMARY_OUTPUT_DIR:
    raise ValueError("IMPACT_SUMMARY_OUTPUT_DIR variable not set.")

# Directory to output plots
PLOT_OUTPUT_DIR = os.environ.get('IMPACT_PLOT_OUTPUT_DIR')
if not PLOT_OUTPUT_DIR:
    raise ValueError("IMPACT_PLOT_OUTPUT_DIR variable not set.")


# Directory for archive files
ARCHIVE_DIR = os.environ.get('IMPACT_ARCHIVE_DIR')
if not ARCHIVE_DIR:
    raise ValueError("ARCHIVE_DIR variable not set.")

# Directory for EPICS snapshot files
SNAPSHOT_DIR = os.environ.get('IMPACT_SNAPSHOT_DIR')
if not SNAPSHOT_DIR:
    raise ValueError("SNAPSHOT_DIR variable not set.")

# Dummy file for distgen
DISTGEN_LASER_FILE = os.environ.get('IMPACT_DISTGEN_LASER_FILE')
if not DISTGEN_LASER_FILE:
    raise ValueError("IMPACT_DISTGEN_LASER_FILE variable not set.")

# Number of processorss
NUM_PROCS = os.environ.get('IMPACT_NUM_PROCS')
if not NUM_PROCS:
    raise ValueError("IMPACT_NUM_PROCS variable not set.")
else:
    NUM_PROCS = int(NUM_PROCS)


# if using sdf:
if HOST == 'sdf':    
    # check that environment variables are configured for execution
    IMPACT_COMMAND = os.environ.get("IMPACT_COMMAND")
    if not IMPACT_COMMAND:
        raise ValueError("IMPACT_COMMAND variable not set.")


    IMPACT_COMMAND_MPI = os.environ.get("IMPACT_COMMAND_MPI")
    if not IMPACT_COMMAND_MPI:
        raise ValueError("IMPACT_COMMAND_MPI variable not set.")


MPI_RUN_CMD = os.environ.get("IMPACT_MPI_RUN_CMD")
if not MPI_RUN_CMD:
    raise ValueError("IMPACT_MPI_RUN_CMD variable not set.")


'/nfs/slac/g/beamphysics/cmayes/GitHub/lume-impact-live-demo/cu_inj-data'

In [9]:
CONFIG0 = {}

# Base settings
SETTINGS0 = {
 'distgen:n_particle': 100_000,   
 'timeout': 10000,
 'header:Nx': 32,
 'header:Ny': 32,
 'header:Nz': 32,
   }

if DEBUG:
    logger.info('DEBUG MODE: Running without space charge for speed. ')
    SETTINGS0['distgen:n_particle'] =1000
    SETTINGS0['total_charge'] = 0
    SETTINGS0['numprocs'] = NUM_PROCS
    
if HOST == 'mcc-simul':
    
    SETTINGS0['numprocs'] = NUM_PROCS
    SETTINGS0['mpi_run'] = 'mpirun -n {n} --use-hwthread-cpus {command_mpi}
    CONFIG0['workdir'] = None
    
elif HOST == 'sdf':
    
     # SDF setup    
    SETTINGS0['numprocs'] = 64
    SETTINGS0['command'] =  impact_command
    SETTINGS0['command_mpi'] =  impact_command_mpi
    SETTINGS0['mpi_run'] =  'salloc --partition ard -N 1 -n {n} /sdf/sw/gcc-4.8.5/openmpi-4.0.4/bin/mpirun -n {n} {command_mpi}'
    
    CONFIG0['workdir'] = os.path.expandvars('$SCRATCH')
    
else:
    raise ValueError(f'Unknown host: {HOST}')
    

# Select: LCLS or FACET

In [11]:
# PV -> Sim conversion table
CSV =  f'pv_mapping/{MODEL}_impact.csv'  


CONFIG0['impact_config']      =  IMPACT_INPUT_FILE
CONFIG0['distgen_input_file'] =  DISTGEN_INPUT_FILE


if MODEL == 'cu_inj'
    VCC_DEVICE = 'CAMR:IN20:186' # LCLS   
    
    DASHBOARD_KWARGS = {'outpath':PLOT_OUTPUT_DIR,
                    'screen1': 'YAG02',
                    'screen2': 'YAG03',
                    'screen3': 'OTR2',
                    'ylim' : (0, 2e-6), # Emittance scale                        
                    'name' : PREFIX
                   }    
    
    SETTINGS0['stop'] = 16.5
    SETTINGS0['distgen:t_dist:length:value'] =  4 * 1.65   #  Inferred pulse stacker FWHM: 4 ps, converted to tukey length
    
elif MODEL == 'f2e_inj':
    VCC_DEVICE = 'CAMR:LT10:900' # FACET-II
    
    DASHBOARD_KWARGS = {'outpath':PLOT_OUTPUT_DIR,
                    'screen1': 'PR10241',
                    'screen2': 'PR10465',
                    'screen3': 'PR10571',
                    'ylim' : (0, 20e-6), # Emittance scale
                    'name' : PREFIX
                   }        
    
    SETTINGS0['distgen:t_dist:length:value'] =  3.65 * 1.65   #  Measured FWHM: 3.65 ps, converted to tukey length
     
else:
    raise

# Set up monitors

In [12]:
DF = pd.read_csv(CSV)#.dropna()

PVLIST = list(DF['device_pv_name'].dropna()) + list(VCC_DEVICE_PV[VCC_DEVICE].values())

#DF.set_index('device_pv_name', inplace=True)
DF

,Variable,bmad_name,device_pv_name,device_min,device_max,scan_min,scan_max,pv_unit,impact_name,impact_factor,impact_description,impact_unit
0,Solenoid,SOL1,SOLN:IN20:121:BACT,0.000,0.550,0.350,0.500,kG*m,SOL1:solenoid_field_scale,5.142724e-01,peak field,T
1,Corrector quad,CQ01,QUAD:IN20:121:BACT,-0.015,0.015,-0.015,0.015,kG,CQ01:b1_gradient,-4.761905e-01,Corrector quad gradient,T/m
2,Skew quad,SQ01,QUAD:IN20:122:BACT,-0.015,0.015,-0.015,0.015,kG,SQ01:b1_gradient,-4.761905e-01,Skew quad gradient,T/m
3,L0A phase,L0A,ACCL:IN20:300:L0A_PDES,-20.000,0.000,-20.000,0.000,deg,L0A_phase:dtheta0_deg,1.000000e+00,Phase relative to on-crest,deg
4,L0B phase,L0B,ACCL:IN20:400:L0B_PDES,-20.000,0.000,-20.000,0.000,deg,L0B_phase:dtheta0_deg,1.000000e+00,Phase relative to on-crest,deg
5,L0B voltage,L0A,ACCL:IN20:300:L0A_ADES,0.000,110.000,50.000,66.000,MV,L0A_scale:voltage,1.000000e+06,Voltage,V
6,L0B voltage,L0B,ACCL:IN20:400:L0B_ADES,0.000,110.000,70.000,70.000,MV,L0B_scale:voltage,1.000000e+06,Voltage,V
7,QA01,QA01,QUAD:IN20:361:BACT,-20.000,20.000,-3.500,-2.750,kG,QA01:b1_gradient,-9.259259e-01,10.8 cm quad,T/m
8,QA02,QA02,QUAD:IN20:371:BACT,-20.000,20.000,2.500,2.900,kG,QA02:b1_gradient,-9.259259e-01,10.8 cm quad,T/m
9,QE01,QE01,QUAD:IN20:425:BACT,-20.000,20.000,-4.000,-1.000,kG,QE01:b1_gradient,-9.259259e-01,10.8 cm quad,T/m


In [13]:
MONITOR = {pvname:epics.PV(pvname) for pvname in PVLIST}
sleep(5)

In [14]:
def get_pvdata():
        
    itime = isotime()
    pvdata =  {k:MONITOR[k].get() for k in MONITOR}
    
    logger.info(f'Acquired settings from EPICS at: {itime}')
    
    for k, v in pvdata.items():
        
        if v is None:
            raise ValueError(f'EPICS get for {k} returned None')
        
        if ':IMAGE' in k.upper():
            if v.ptp() < 128:
                v = v.astype(np.int8) # Downcast preeptively 
            if v.ptp() == 0:
                raise ValueError(f'EPICS get for {k} has zero extent')
            pvdata[k] = v
    return pvdata, itime
PVDATA, ITIME = get_pvdata()
PVDATA, ITIME

Acquired settings from EPICS at: 2021-12-08T20:49:45-08:00


({'SOLN:IN20:121:BACT': 0.47087147970847476,
  'QUAD:IN20:121:BACT': 0.0033037103800510736,
  'QUAD:IN20:122:BACT': 0.001458067452619012,
  'ACCL:IN20:300:L0A_PDES': 0.0,
  'ACCL:IN20:400:L0B_PDES': -2.5,
  'ACCL:IN20:300:L0A_ADES': 58.0,
  'ACCL:IN20:400:L0B_ADES': 69.59528554537978,
  'QUAD:IN20:361:BACT': -3.329082663747593,
  'QUAD:IN20:371:BACT': 2.6212450399657987,
  'QUAD:IN20:425:BACT': -1.7692845482105874,
  'QUAD:IN20:441:BACT': -0.05010523276547739,
  'QUAD:IN20:511:BACT': 2.949197553683145,
  'QUAD:IN20:525:BACT': -2.9990579036734446,
  'BPMS:IN20:221:TMIT': 4600201.0,
  'CAMR:IN20:186:IMAGE': array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  'CAMR:IN20:186:N_OF_COL': 640,
  'CAMR:IN20:186:N_OF_ROW': 480,
  'CAMR:IN20:186:RESOLUTION': 9.0,
  'CAMR:IN20:186:RESOLUTION.EGU': 'um'},
 '2021-12-08T20:49:45-08:00')

In [15]:
# Saving and loading
def save_pvdata(filename, pvdata, isotime):
    with h5py.File(filename, 'w') as h5:
        h5.attrs['isotime'] = np.string_(isotime)
        for k, v in pvdata.items():
            if isinstance(v, str):
                v =  np.string_(v)
            h5[k] = v 
def load_pvdata(filename):
    pvdata = {}
    with h5py.File(filename, 'r') as h5:
        isotime = h5.attrs['isotime']
        for k in h5:
            v = np.array(h5[k])        
            if v.dtype.char == 'S':
                v = str(v.astype(str))
            pvdata[k] = v
            
    return pvdata, isotime
save_pvdata('test.h5', PVDATA, isotime()) 
load_pvdata('test.h5')

({'ACCL:IN20:300:L0A_ADES': array(58.),
  'ACCL:IN20:300:L0A_PDES': array(0.),
  'ACCL:IN20:400:L0B_ADES': array(69.59528555),
  'ACCL:IN20:400:L0B_PDES': array(-2.5),
  'BPMS:IN20:221:TMIT': array(4600201.),
  'CAMR:IN20:186:IMAGE': array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  'CAMR:IN20:186:N_OF_COL': array(640),
  'CAMR:IN20:186:N_OF_ROW': array(480),
  'CAMR:IN20:186:RESOLUTION': array(9.),
  'CAMR:IN20:186:RESOLUTION.EGU': 'um',
  'QUAD:IN20:121:BACT': array(0.00330371),
  'QUAD:IN20:122:BACT': array(0.00145807),
  'QUAD:IN20:361:BACT': array(-3.32908266),
  'QUAD:IN20:371:BACT': array(2.62124504),
  'QUAD:IN20:425:BACT': array(-1.76928455),
  'QUAD:IN20:441:BACT': array(-0.05010523),
  'QUAD:IN20:511:BACT': array(2.94919755),
  'QUAD:IN20:525:BACT': array(-2.9990579),
  'SOLN:IN20:121:BACT': array(0.47087148)},
 b'2021-12-08T20:49:45-08:00')

In [16]:
#while True:
#    get_pvdata()

# EPICS -> Simulation settings

In [17]:
def get_live_settings(csv, base_settings={}, snapshot_dir=None):
    """
    Fetches live settings for all devices in the CSV table, and translates them to simulation inputs
     
    """
    df = DF[DF['device_pv_name'].notna()]
    assert len(df) > 0, 'Empty dataframe!'
    
    pv_names = list(df['device_pv_name'])

    pvdata, itime = get_pvdata()
    
    if snapshot_dir:
        filename = os.path.abspath(os.path.join(snapshot_dir, f'{MODEL}-snapshot-{itime}.h5'))
        logger.info(f'EPICS shapshot written: {filename}')
        save_pvdata(filename, pvdata, itime)
        # DEBUG: check readback
        #pvdata, itime = load_pvdata(filename)
        
    #df['pv_value'] = epics.caget_many(pv_names)
    df['pv_value'] = [pvdata[k] for k in pv_names]
    
    # Assign impact
    df['impact_value'] = df['impact_factor']*df['pv_value'] 

    # Collect settings
    settings = base_settings.copy()
    settings.update(dict(zip(df['impact_name'], df['impact_value'])))

    # VCC image
    dfile, img, cutimg = get_live_distgen_xy_dist(filename=DISTGEN_LASER_FILE, vcc_device=VCC_DEVICE, pvdata=pvdata)
    settings['distgen:xy_dist:file'] = dfile
    
    return settings, df, img, cutimg, itime

res = get_live_settings(CSV, SETTINGS0, snapshot_dir='.')
#DF[['Variable', 'bmad_name', 'pv_value','pv_unit',  'device_min', 'device_max',  'impact_name', 'impact_factor', 'impact_unit',
#        'impact_description',  'impact_value']]
res[1]

Acquired settings from EPICS at: 2021-12-08T20:49:45-08:00
EPICS shapshot written: /nfs/slac/g/beamphysics/cmayes/GitHub/lume-impact-live-demo/cu_inj-snapshot-2021-12-08T20:49:45-08:00.h5


,Variable,bmad_name,device_pv_name,device_min,device_max,scan_min,scan_max,pv_unit,impact_name,impact_factor,impact_description,impact_unit,pv_value,impact_value
0,Solenoid,SOL1,SOLN:IN20:121:BACT,0.000,0.550,0.350,0.500,kG*m,SOL1:solenoid_field_scale,5.142724e-01,peak field,T,4.708715e-01,2.421562e-01
1,Corrector quad,CQ01,QUAD:IN20:121:BACT,-0.015,0.015,-0.015,0.015,kG,CQ01:b1_gradient,-4.761905e-01,Corrector quad gradient,T/m,3.303710e-03,-1.573195e-03
2,Skew quad,SQ01,QUAD:IN20:122:BACT,-0.015,0.015,-0.015,0.015,kG,SQ01:b1_gradient,-4.761905e-01,Skew quad gradient,T/m,1.458067e-03,-6.943178e-04
3,L0A phase,L0A,ACCL:IN20:300:L0A_PDES,-20.000,0.000,-20.000,0.000,deg,L0A_phase:dtheta0_deg,1.000000e+00,Phase relative to on-crest,deg,0.000000e+00,0.000000e+00
4,L0B phase,L0B,ACCL:IN20:400:L0B_PDES,-20.000,0.000,-20.000,0.000,deg,L0B_phase:dtheta0_deg,1.000000e+00,Phase relative to on-crest,deg,-2.500000e+00,-2.500000e+00
5,L0B voltage,L0A,ACCL:IN20:300:L0A_ADES,0.000,110.000,50.000,66.000,MV,L0A_scale:voltage,1.000000e+06,Voltage,V,5.800000e+01,5.800000e+07
6,L0B voltage,L0B,ACCL:IN20:400:L0B_ADES,0.000,110.000,70.000,70.000,MV,L0B_scale:voltage,1.000000e+06,Voltage,V,6.959510e+01,6.959510e+07
7,QA01,QA01,QUAD:IN20:361:BACT,-20.000,20.000,-3.500,-2.750,kG,QA01:b1_gradient,-9.259259e-01,10.8 cm quad,T/m,-3.329083e+00,3.082484e+00
8,QA02,QA02,QUAD:IN20:371:BACT,-20.000,20.000,2.500,2.900,kG,QA02:b1_gradient,-9.259259e-01,10.8 cm quad,T/m,2.621245e+00,-2.427079e+00
9,QE01,QE01,QUAD:IN20:425:BACT,-20.000,20.000,-4.000,-1.000,kG,QE01:b1_gradient,-9.259259e-01,10.8 cm quad,T/m,-1.769285e+00,1.638226e+00


In [18]:
gfile = CONFIG0['distgen_input_file']
from distgen import Generator
#fout = res[0]
G = Generator(gfile)
#G['xy_dist:file'] =  DISTGEN_LASER_FILE #'distgen_laser.txt'
G['xy_dist:file'] = res[0]['distgen:xy_dist:file'] 
G['n_particle'] = 100000
G.run()
G.particles.plot('x', 'y', figsize=(5,5))

In [19]:
#CONFIG0

In [20]:
#%%time
#I0 = run_impact_with_distgen(LIVE_SETTINGS,  WORKDIR=WORKDIR, **CONFIG0, verbose=True )

In [21]:
DO_TIMING = False

if DO_TIMING:
    import numpy as np
    import time
    results = []
    tlist = []
    nlist = 2**np.arange(1,8, 1)[::-1]
    for n in nlist:
        t1 = time.time()
        LIVE_SETTINGS['numprocs'] = n
        print(f'running wit {n}')
        result = run_impact_with_distgen(LIVE_SETTINGS, **CONFIG0, verbose=False )
        results.append(result)
        dt = time.time() - t1
        tlist.append(dt)
        print(n, dt)     
        
    tlist, nlist        

In [22]:
# %matplotlib inline
# I0.plot(['norm_emit_x'], y2=['sigma_x'], ylim=(0, 10e-6), ylim2=(0,2e-3), figsize=(16,9))

In [23]:
#I0.initial_particles.plot('x', 'y')

# Get live values, run Impact-T, make dashboard

In [24]:
# Patch this into the function below for the dashboard creation
def my_merit(impact_object, itime):
    # Collect standard output statistics
    merit0 = default_impact_merit(impact_object)
    # Make the dashboard from the evaluated object
    plot_file = make_dashboard(impact_object, itime=itime, **DASHBOARD_KWARGS)
    #print('Dashboard written:', plot_file)
    logger.info(f'Dashboard written: {plot_file}')
    
    # Assign extra info
    merit0['plot_file'] = plot_file    
    merit0['isotime'] = itime
    
    # Clear any buffers
    plt.close('all')

    return merit0

In [25]:
#my_merit(I0, '123')

In [27]:
# %%time
# # Here are the results
# result = run1()
# result.keys()

In [26]:
def run1():
    dat = {}
    
    # Acquire settings
    mysettings, df, img, cutimg, itime = get_live_settings(CSV, SETTINGS0, snapshot_dir=SNAPSHOT_DIR)
        
    dat['isotime'] = itime
    
    # Record inputs
    dat['inputs'] = mysettings
    dat['config'] = CONFIG0
    dat['pv_mapping_dataframe'] = df.to_dict()
    
    logger.info(f'Running evaluate_impact_with_distgen...')

    t0 = time()
    dat['outputs'] = evaluate_impact_with_distgen(mysettings,
                                       merit_f=lambda x: my_merit(x, itime),
                                       archive_path=ARCHIVE_DIR,
                                       **CONFIG0, verbose=False )
    logger.info(f'...finished in {(time()-t0)/60:.1f} min')
    fname = fname=f'{SUMMARY_OUTPUT_DIR}/{PREFIX}-{itime}.json'
    json.dump(dat, open(fname, 'w'))
    #print('Written:', fname)
    logger.info(f'Output written: {fname}')
    return dat
    

In [28]:
# Basic config
#result['config']

In [29]:
# Simulation inputs
#result['inputs']

In [30]:
# Simulation outputs
#result['outputs']

# Show the plot 

In [31]:
# from IPython.display import Image
# Image(filename=result['outputs']['plot_file']) 

# loop it


In [32]:
if __name__ == '__main__':
    while True:
        try:
            run1()
        except:
            sleep(10)
            logger.info('Something BAD happened. Sleeping for 10 s ...')